<!--NAVIGATION-->
< [Vorherige Lektion](06_Shell_Shell-Skripte.ipynb) || [Verständnisfragen zu dieser Lektion](Verstaendnisfragen_zu_Lektion_07_Dateiinhalte_finden_und_nutzen.ipynb) |

# Dateiinhalte finden und nutzen

## Was behandeln wir in diesem Notebook / Lernziele

<div class="alert alert-success">
    
<b>Fragestellung:</b>
<ul>
    <li> Wie kann ich existierende <code>Unix</code>-Programme kombinieren, um neue und komplexere Aufgaben zu erledigen?</li>
</ul>

<b>Aspekte der Fragestellung:</b>
<ul>    
    <li> Was ist die <i>Kommandosubstitution</i> und wie benutze ich sie?</li>
    <li> Wie finde ich bestimmte Dateiinhalte in Textdateien?</li>
</ul>

<b> Zeitaufwand für diese Lektion: </b> 
<ul>
    <li> Durcharbeiten des Textes: 30 min</li>
    <li> Verständnisfragen: 20 min</li>
</ul>
</div>

In dieser letzten Lektion wollen wir uns mit der Kommandosubstitution noch eine Technik anschauen, um sehr allgemeine, benutzerdefinierte Listen für Befehle und Schleifenkonstrukte zu kreieren. Die Kommandosubstitution erweitert und verallgemeinert die Wildcards, welche es uns auf einfach Weise erlauben, große Dateilisten zu erstellen und in Kommandos zu verwenden.

Der `grep`-Filter, welcher Dateien nach bestimmten Inhalten filtern kann, schließt diese Lektion und das Tutorial ab.

## Die Kommandosubstitution

Bisher haben wir längere Listen aus Dateien für Kommandos oder Shellskripte effektiv mit Wildcards erstellt. Die Wildcards `*` und `?` erlauben es uns, mit sehr wenig Tippaufwand lange Dateilisten nach Mustern in den Dateinamen zu erstellen. Oft möchte man jedoch Dateilisten (oder Listen anderer Objekte) nach völlig anderen Kriterien zusammenstellen, z.B aufgrund des Dateiinhalts. Eine Möglichkeit, solche Listen ähnlich effizient wie mit Wildcards zu erzeugen, ist die so genannte *Kommandosubstitution*.
Sehen wir uns ein konkretes Beispiel an, in dem Annika die Integrität ihrer Beobachtungen intensiver testen möchte: 

In [ ]:
cd ~/Bachelor_Arbeit/Beobachtungen
ls -F

Nachdem Annika, wie in [Lektion 5](05_Shell_for_Schleife.ipynb#Statistik_Beobachtungen), Statistiken aller Beobachtungen berechnet hat, möchte sie weitergehende Tests durchführen. Sie weiß, dass jede der ungefähr 500 Dateien in den Ordnern `raw` und `processed` genau 100 Zeilen enthalten sollte. Sie testet zunächst diese Annahme für die Daten im Verzeichnis `raw`:

In [ ]:
# Teste, ob alle Dateien in Verzeichnis raw genau 100 Zeilen haben
wc -l raw/*.txt

Das sieht beim schnellen Überfliegen gut aus. Für einen kompakteren Überblick bei den vielen Dateien schreibt Annika aber noch eine erweiterte Pipeline, die nur anzeigt, wieviele Dateien es mit wievielen Zeilen gibt:

In [ ]:
# Teste, ob alle Dateien in Verzeichnis raw genau 100 Zeilen haben
# Folgende Pipeline zeigt, wie oft jeder Eintrag der ersten Spalte des Befehls
# wc -l raw/*.txt (die Anzahl der Zeilen in den Dateien) vorkommt.
# Im Idealfall sollte nur die Zahl 100 in dieser ersten Spalte
# vorhanden sein!
wc -l raw/*.txt | awk '{print $1}' | sort | uniq -c

Die Ausgabe bedeutet, dass es 481 Dateien mit je 100 Zeilen gibt. Die zweite Zeile der Ausgabe, `1 48100`, kommt von der Gesamtbilanz, die `wc` bei der Eingabe mehrerer Dateien zusätzlich angibt. Nun dasselbe für den `processed`-Ordner:

In [ ]:
wc -l processed/*.txt | awk '{print $1}' | sort | uniq -c

Hier gibt es offenbar ein Problem und wir haben Dateien mit 100, 95 und 90 Zeilen. Annika möchte zunächst die Dateien mit 95 Zeilen genauer untersuchen und sie hierzu in einen separaten Ordner kopieren. Sie erstellt sich zunächst eine Liste der fraglichen Dateien:

In [ ]:
# liste alle Dateien in processed, die genau 95 Zeilen haben
wc -l processed/*.txt | awk '($1 == 95) {print $2}'

Mit der Syntax `awk '($1 == 95) {print $2}'` listet `awk` die zweite Spalte aller Einträge, die in der ersten Spalte einen numerischen Wert von 95 haben. Diese Liste sichern wir in einer Datei:

In [ ]:
wc -l processed/*.txt | awk '($1 == 95) {print $2}' > problem_95.txt
cat problem_95.txt

Nun kopieren wir die in `problem_95.txt` gelisteten Dateien in ein Verzeichnis `problemfaelle`:

In [ ]:
mkdir problemfaelle

In [ ]:
cp $(cat problem_95.txt) problemfaelle
ls problemfaelle

Bei dem `cp $(cat problem_95.txt) problemfaelle`-Befehl passiert folgendens: Sieht die Shell in einem Befehl das Konstrukt `$(...)`, so wird zunächst der darin eingebettete Befehl ausgeführt. Danach wird das gesamte Konstrukt `$(...)` durch *die Ausgabe dieses Befehls ersetzt* und die Abarbeitung der Kommandozeile fortgesetzt.

Im vorliegenden Fall liefert `cat problem_95.txt` die Ausgabe `processed/695833p.txt ... processed/774957p.txt`. Hiermit wird für den Befehl `cp $(cat problem_95.txt) problemfaelle` nach der *Kommandosubstitution* effektiv der Befehl `cp processed/695833p.txt ... processed/774957p.txt problemfaelle` ausgeführt. Dies ist genau die gewünschte Kopieroperation.

Die Kommandosubstitution verhält sich damit ähnlich der Behandlung eines Befehls mit den Wildcards `*` und `?`. Somit können wir hier effektiv *eigene* Wildcards mit *beliebigen* Befehlen erzeugen!

Genau, wie wir z.B. in [Lektion 5](05_Shell_for_Schleife.ipynb#for-statistik) Wildcards für Listen in `for`-Schleifen verwendet haben, können wir dies auch mit der Kommandosubstitution tun. Annika könnte daher folgenden Befehl verwenden, um Statistiken nur von den problematischen Daten zu errechnen:

In [ ]:
# Berechne Statistik für problematische Felder,
# die in 'problem_95.txt' gelistet sind:
for BEOBACHTUNG in $(cat problem_95.txt)
do
  echo "Ich arbeite an ${BEOBACHTUNG}"
  python3 calc_stats.py ${BEOBACHTUNG}
done

## Der `grep`-Filter (Dateiinhalte in Textdateien finden)

Wir wollen uns in diesem Abschnitt noch einen weiteren, sehr wichtigen, Filter ansehen, der sehr häufig in Pipelines und für *eigene Wildcards* mit der Kommandosubtitution verwendet wird. Es handelt sich um den `grep`-Befehl. Genau wie der Begriff *`Google`-Suche* (oder *googeln*) inzwischen ein Synonym für *Webseiten mit bestimmten Inhalten finden* ist, ist `grep` unter `Unix`-Nutzern ein Synonym für *bestimmte Inhalte in Textdateien finden*. Der Name des Befehls ist eine Abkürzung für  *global/regular expression/print*. Die Folge `grep` wurde erstmals in den 1970er Jahren in dem Texteditor `vi` benutzt, um gewissen Textstellen zu finden. Hieraus hat sich später ein eigenständiges `Unix`-Programm entwickelt.

Wir behandeln hier lediglich die einfachste Form des Kommandos, die aber für den Anfang für Sie vollkommen ausreichend ist.

Wir hatten uns bereits in [Lektion 4](04_Shell_Pipelines_und_Filter.ipynb#Annika_Test_Beobachtungen) mit der Datei `Beobachtungs_Logdatei.txt` beschäftigt, welche Informationen über Annikas Beobachtungen enthält:
<a id='Logdatei'></a>

In [ ]:
cd ~/Bachelor_Arbeit/Beobachtungen
head Beobachtungs_Logdatei.txt

Die für uns wichtigen Spalten sind hier die erste mit der Identifikation der Beobachtung, die zweite Spalte aus Beobachtungsfeld (`D1-D4`) und Beobachtungskonfiguration (`u`, `g`, `r`, `i` oder `z`) und die vierte Spalte, die den Beobachtungszeitpunkt (die Anzahl der Tage nach einem bestimmten Referenzdatum) enthält.

Annika möchte Statistiken für Beobachtungen bestimmter Tage berechnen und daher gerne Daten nach der vierten Spalte der Datei `Beobachtungs_Logdatei.txt` selektieren. Ihr Betreuer empfiehlt ihr ein spezielles Augenmerk auf die Tage 1664 und 1673 zu werfen. Mit folgendem Befehl wendet sich Annika zunächst dem Tag 1664 zu:

In [ ]:
grep "1664" Beobachtungs_Logdatei.txt

Der Befehl `grep "1664" Beobachtungs_Logdatei.txt` zeigt alle Zeilen der Datei `Beobachtungs_Logdatei.txt`, welche die Zeichenkette `1664` enthalten und er demonstriert bereits die grundlegende Funktion des Programms `grep`. Das Kommando zeigt einfach alle Zeilen, die eine entsprechende Zeichenkette enthalten. Die Zeichenkette selber sollte in doppelte Anführungsstriche gesetzt sein.

Hiermit kann Annika mit bereits bekannten Techniken ihre benötigten Statistiken für Beobachtungen des Tages 1664 berechnen:    

In [ ]:
# selektiere alle Beobachtungen des Tages 1664 und
# speichere sie in eine Datei daten_1664.txt
grep "1664" Beobachtungs_Logdatei.txt | awk '{print $1}' > daten_1664.txt
cat daten_1664.txt

In [ ]:
# Berechne jetzt die notwendigen Statistiken
for BEOBACHTUNG in $(cat daten_1664.txt)
do
  echo "Ich arbeite an ${BEOBACHTUNG}"
  # Man beachte, dass wir im folgenden Befehl
  # das Verzeichnis 'processed' und die Dateiendung
  # '.txt' explizit angeben müssen, da die Datei
  # daten_1664.txt nur die Beobachtungs-IDs enthält!
  python3 calc_stats.py processed/${BEOBACHTUNG}.txt
done

Annika wendet sich nun dem Tag 1673 zu und besorgt sich zunächst, wie oben, eine Liste der entsprechenden Daten:

In [ ]:
grep "1673" Beobachtungs_Logdatei.txt

Hier scheint `grep` ein verkehrtes Ergebnis zu liefern, da auch ein Datum des Tages `1926` in der Liste vorhanden ist. Das Problem ist, dass die Zeichenkette `1673` in der Beobachtungs-ID `741673p` enthalten ist und die Zeile damit von `grep` korrekt, aber für Annika unerwünscht, selektiert wird. Dieser Effekt kann mit folgenden Befehl beseitigt werden:

In [ ]:
grep -w "1673" Beobachtungs_Logdatei.txt

Die `grep`-Option `-w (word)` selektiert nur Zeilen, welche die Argumentzeichenkette als *eigenes Wort* enthalten und sie nicht nur in einer längeren Zeichenkette eingebettet enthalten. Hiermit kann Annika den Tag `1673` analog zu Tag `1664` analysieren: 

In [ ]:
grep -w "1673" Beobachtungs_Logdatei.txt | awk '{print $1}' > daten_1673.txt
for BEOBACHTUNG in $(cat daten_1673.txt)
do
  echo "Ich arbeite an ${BEOBACHTUNG}"
  python3 calc_stats.py processed/${BEOBACHTUNG}.txt
done

Eine sehr häufig benutzte weitere Option für `grep` ist `-v (vice versa)`, welche alle Zeilen einer Datei listet, die die Argumentzeichenkette *nicht* enthalten. Hiermit können wir sehr einfach ein etwas lästiges Problem lösen, das uns weiter oben in dieser Lektion begegnet ist:

In [ ]:
# Liste die Zeilen mehrerer Dateien, hier
# daten_1664.txt und daten_1673.txt
wc -l daten_1664.txt daten_1673.txt

Das Kommando `wc` gibt bei mehreren übergebenen Dateien immer eine Gesamtbilanz aller Dateien zum Schluss. Diese ist nicht immer erwünscht. Mit einem `grep`-Befehl ist dies einfach zu beheben:

In [ ]:
# Entferne die Gesamtvilanz von 'wc',
# indem aus der Ausgabe die Zeile mit 'total'
# entfernt wird:
wc -l daten_1664.txt daten_1673.txt | grep -v "total"

Erkennungsmerkmal der Gesamtbilanz in der Ausgabe des `wc`-Befehls ist das Wort `total`. Der Filter `grep -v "total"` entfernt genau diese Zeile.

`grep` besitzt noch *recht viele* weitere Optionen, mit denen die Suche verfeinert und beeinflusst werden kann. In dieser ersten Einführung wollen wir es aber bei `-v` und `-w` bewenden lassen.

# Nachwort

Das Ziel dieser Lektionen war es, Ihnen die Benutzung des `Unix`-Kommandozeileninterfaces näherzubringen. Die Shell stammt aus den Anfangszeiten der Entwicklung von Computern und ist heute deutlich älter als die allermeisten ihrer Nutzer. Der Grund, dass sie trotz ausgefeilter und einfach zu nutzender GUIs vor allem im Bereich der Naturwissenschaft immer noch benutzt wird, ist ihre hohe Effizienz bei der Automatisierung von Prozessen. 

Nutzer einer Shell können interaktiv Programme nutzen und nach ausgiebigen Tests Prozesse ohne manuelle Interaktion auf beliebige Datenmengen anwenden und Ergebnisse später ohne großen Aufwand reproduzieren. Während grafische Benutzeroberflächen in der interaktiven Nutzung eines Computers der Shell heute oft überlegen ist, haben diese der Shell im Bereich der Automatisierung sehr wenig entgegenzusetzen. Für die Automatisierung von Datenanalysen und die Reproduzierbarkeit von Ergebnissen ist die Shell nach wie vor die produktivste Umgebung, die bisher erfunden wurde.

Das Hauptziel dieses Tutorials war es, Ihnen die allerwichtigsten Kenntnisse für grundlegende Arbeiten mit der `Unix`-Shell zu vermitteln. Das Hauptaugenmerk lag auf den Techniken und Ideen für eine effiziente Automatisierung von Analyseprozessen. Wir haben andere wichtige `Unix`-Themen, die mit diesem Ziel nicht unmittelbar in Verbindung stehen (z.B. das *`Unix`-Rechtesystem*, *Datentransfer und Remote-Arbeiten auf Fremdrechnern*, *Prozesskontrolle* oder konkrete `Unix`-Programme) hier nicht behandelt. Mit dem erworbenem Wissen werden Sie aber keine Probleme haben, sich diese Kenntnisse bei Bedarf aus dem Internet oder aus der Fachliteratur anzueignen und bei ihrer Arbeit mit der `Unix`-Shell anzuwenden.

<div class="alert alert-success">
<b>Zum Mitnehmen</b>
<ul>
    <li> Die Kommandosubtitution <code>&#36;(kommando)</code> ersetzt ihr Vorkommen durch die Ausgabe von <code>kommando</code>. Dadurch ist es möglich, die Ausgabe eines Programms als Argumente eines anderen zu verwenden.</li>
    <li> <code>grep "zeichenkette"</code> zeigt Zeilen von Textdateien, welche <code>zeichenkette</code> enthalten.</li>
</ul>
</div>

<!--NAVIGATION-->
< [Vorherige Lektion](06_Shell_Shell-Skripte.ipynb) || [Verständnisfragen zu dieser Lektion](Verstaendnisfragen_zu_Lektion_07_Dateiinhalte_finden_und_nutzen.ipynb) |